In [ ]:
import os
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
import cv2
import matplotlib.pyplot as plt
import seaborn as sns


In [ ]:
import kagglehub
# Download latest version
path = kagglehub.dataset_download("masoudnickparvar/brain-tumor-mri-dataset")

print("Path to dataset files:", path)

Using Colab cache for faster access to the 'brain-tumor-mri-dataset' dataset.
Path to dataset files: /kaggle/input/brain-tumor-mri-dataset


In [ ]:
def load_dataset(split="Training"):
    data = []
    split_path = os.path.join(path, split)
    for label in os.listdir(split_path):
        label_path = os.path.join(split_path, label)
        if os.path.isdir(label_path):
            for img in os.listdir(label_path):
                data.append([os.path.join(label_path, img), label])
    return pd.DataFrame(data, columns=["Image", "label"])

In [ ]:
train_df = load_dataset("Training")
test_df = load_dataset("Testing")

In [ ]:
train_df

,Image,label
0,/kaggle/input/brain-tumor-mri-dataset/Training...,pituitary
1,/kaggle/input/brain-tumor-mri-dataset/Training...,pituitary
2,/kaggle/input/brain-tumor-mri-dataset/Training...,pituitary
3,/kaggle/input/brain-tumor-mri-dataset/Training...,pituitary
4,/kaggle/input/brain-tumor-mri-dataset/Training...,pituitary
...,...,...
5707,/kaggle/input/brain-tumor-mri-dataset/Training...,glioma
5708,/kaggle/input/brain-tumor-mri-dataset/Training...,glioma
5709,/kaggle/input/brain-tumor-mri-dataset/Training...,glioma
5710,/kaggle/input/brain-tumor-mri-dataset/Training...,glioma


In [ ]:
test_df

,Image,label
0,/kaggle/input/brain-tumor-mri-dataset/Testing/...,pituitary
1,/kaggle/input/brain-tumor-mri-dataset/Testing/...,pituitary
2,/kaggle/input/brain-tumor-mri-dataset/Testing/...,pituitary
3,/kaggle/input/brain-tumor-mri-dataset/Testing/...,pituitary
4,/kaggle/input/brain-tumor-mri-dataset/Testing/...,pituitary
...,...,...
1306,/kaggle/input/brain-tumor-mri-dataset/Testing/...,glioma
1307,/kaggle/input/brain-tumor-mri-dataset/Testing/...,glioma
1308,/kaggle/input/brain-tumor-mri-dataset/Testing/...,glioma
1309,/kaggle/input/brain-tumor-mri-dataset/Testing/...,glioma


In [ ]:
train_df["label"].value_counts()


,count
label,
notumor,1595
pituitary,1457
meningioma,1339
glioma,1321


In [ ]:
test_df["label"].value_counts()

,count
label,
notumor,405
meningioma,306
pituitary,300
glioma,300


In [ ]:
Valid_df , Test_df = train_test_split(test_df , train_size = 0.5 ,shuffle= True, random_state=42)

In [ ]:
Valid_df["label"].value_counts()

,count
label,
notumor,197
meningioma,158
glioma,151
pituitary,149


In [ ]:
Test_df["label"].value_counts()

,count
label,
notumor,208
pituitary,151
glioma,149
meningioma,148


In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=15,
    zoom_range=0.1,
    horizontal_flip=True,
    brightness_range=[0.8, 1.2],
)
valid_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)


In [ ]:
train_generator = train_datagen.flow_from_dataframe(
    dataframe=train_df,
    x_col = "Image",
    y_col = "label",
    target_size = (128,128),
    class_mode = 'categorical',
    color_mode='grayscale',
    shuffle=True,
    batch_size=64
)

Found 5712 validated image filenames belonging to 4 classes.


In [ ]:
valid_generator = valid_datagen.flow_from_dataframe(
    dataframe=Valid_df,
    x_col = "Image",
    y_col = "label",
    target_size = (128,128),
    class_mode = 'categorical',
    color_mode='grayscale',
    shuffle=False,
    batch_size=64
)

Found 655 validated image filenames belonging to 4 classes.


In [ ]:
test_generator = test_datagen.flow_from_dataframe(
    dataframe=Test_df,
    x_col = "Image",
    y_col = "label",
    target_size = (128,128),
    class_mode = 'categorical',
    color_mode='grayscale',
    shuffle=False,
    batch_size=64
)

Found 656 validated image filenames belonging to 4 classes.


## Model

In [ ]:
model = Sequential([


    Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 1)),
    MaxPooling2D(2, 2),


    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),


    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),


    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),


    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.4),
    Dense(4, activation='softmax')
])

In [ ]:
model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [ ]:
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

In [ ]:
history = model.fit(
    train_generator,
    epochs=50,
    validation_data=valid_generator,
    callbacks=[early_stopping]
)

Epoch 1/50
90/90 ━━━━━━━━━━━━━━━━━━━━ 30s 278ms/step - accuracy: 0.4959 - loss: 1.1135 - val_accuracy: 0.6992 - val_loss: 0.7364
Epoch 2/50
90/90 ━━━━━━━━━━━━━━━━━━━━ 33s 224ms/step - accuracy: 0.7230 - loss: 0.6852 - val_accuracy: 0.7496 - val_loss: 0.6794
Epoch 3/50
90/90 ━━━━━━━━━━━━━━━━━━━━ 21s 230ms/step - accuracy: 0.7673 - loss: 0.5619 - val_accuracy: 0.7832 - val_loss: 0.5588
Epoch 4/50
90/90 ━━━━━━━━━━━━━━━━━━━━ 20s 225ms/step - accuracy: 0.7975 - loss: 0.4929 - val_accuracy: 0.7695 - val_loss: 0.5088
Epoch 5/50
90/90 ━━━━━━━━━━━━━━━━━━━━ 20s 226ms/step - accuracy: 0.8217 - loss: 0.4491 - val_accuracy: 0.8046 - val_loss: 0.4945
Epoch 6/50
90/90 ━━━━━━━━━━━━━━━━━━━━ 20s 224ms/step - accuracy: 0.8457 - loss: 0.3874 - val_accuracy: 0.8489 - val_loss: 0.3513
Epoch 7/50
90/90 ━━━━━━━━━━━━━━━━━━━━ 21s 229ms/step - accuracy: 0.8570 - loss: 0.3536 - val_accuracy: 0.8366 - val_loss: 0.4282
Epoch 8/50
90/90 ━━━━━━━━━━━━━━━━━━━━ 20s 222ms/step - accuracy: 0.8863 - loss: 0.3081 - val_accu

In [ ]:
test_loss,test_acc=model.evaluate(test_generator)

11/11 ━━━━━━━━━━━━━━━━━━━━ 5s 437ms/step - accuracy: 0.9731 - loss: 0.0615


In [ ]:
print(f"test_loss : {test_loss}       test_acc : {test_acc} ")

test_loss : 0.06049427017569542       test_acc : 0.9756097793579102 


In [ ]:
model.save("MRI_Class.keras")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
class_dict = train_generator.class_indices

In [ ]:
class_dict

{'glioma': 0, 'meningioma': 1, 'notumor': 2, 'pituitary': 3}